# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Построим модель с максимально большим значением *accuracy*. 

**Импортируем необходимые библиотеки**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

## Откройте и изучите файл

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Предобработка данных не понадобится — мы её уже сделали.

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

- сalls — количество звонков,

- minutes — суммарная длительность звонков в минутах,

- messages — количество sms-сообщений,

- mb_used — израсходованный интернет-трафик в Мб,

- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

В Dataframe 3214 записей, это очень хороший показатель для исследования!

## Разбейте данные на выборки

In [5]:
#features содержит все признаки (характеристики), 
#которые будут использоваться для обучения модели, 
#а target — целевой признак, который модель будет пытаться предсказать.
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1.

**Будем использовать функцию train_test_split(), чтобы разделить исходные данные (features и target) на обучающую выборку (features_train и target_train) и еще одну выборку, которую мы назовем "промежуточной" (features_test и target_test). Значение test_size=0.4 указывает, что 40% данных будут в этой "промежуточной" выборке, а 60% данных составят обучающую выборку.**

In [6]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.4, 
                                                                            random_state=12345
                                                                           ) 

**Затем мы продолжим делить "промежуточную" выборку на валидационную и тестовую выборки:**

In [7]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, 
                                                                            target_test, 
                                                                            test_size=0.5, 
                                                                            random_state=12345
                                                                           ) 

**Поскольку мы использовали test_size=0.5, "промежуточная" выборка была поровну разделена на валидационную (features_valid и target_valid) и тестовую (features_test и target_test) выборки. Это означает, что размер валидационной и тестовой выборки равен 20% от размера исходных данных (0.4 * 0.5 = 0.2).**

Можем проверить размеры этих выборок, чтобы убедиться в правильности разбиения:

In [8]:
print("Обучающая выборка:")
print(features_train.shape)
print(target_train.shape)

print("\nВалидационная выборка:")
print(features_valid.shape)
print(target_valid.shape)

print("\nТестовая выборка:")
print(features_test.shape)
print(target_test.shape)

Обучающая выборка:
(1928, 4)
(1928,)

Валидационная выборка:
(643, 4)
(643,)

Тестовая выборка:
(643, 4)
(643,)


**В результате разбиения мы получили:**

1. Обучающую выборку размером 60% от исходных данных.
2. Валидационную выборку размером 20% от исходных данных.
3. Тестовую выборку размером 20% от исходных данных.

## Исследуйте модели

**`DecisionTreeClassifier` - это структура данных для классификации деревом решений.**

In [9]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) # обучим модель
    predictions = model.predict(features_valid) # получим предсказания модели
    result = accuracy_score(predictions, target_valid) # посчитаем качество модели
    if result > best_result:
        best_model = model
        best_result = result
        best_decisiontreeclassifier_depth = depth
        
print("Accuracy лучшей модели:", best_result, "с гиперпараметром max_depth: =",best_decisiontreeclassifier_depth)

Accuracy лучшей модели: 0.7853810264385692 с гиперпараметром max_depth: = 3


**`RandomForestClassifier`**

**Ещё один из алгоритмов классификации - случайный лес (random forest). Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Случайный лес помогает улучшить результат предсказания и избежать переобучения.**

In [10]:
best_model_randomforestclassifier = None
best_result_randomforestclassifier = 0
for est in range(1, 11):
    for depth in range(1, 11):
        model_randomforestclassifier = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) # обучим модель с заданным количеством деревьев
        model_randomforestclassifier.fit(features_train, target_train) # обучим модель на тренировочной выборке
        result_randomforestclassifier = model_randomforestclassifier.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
        if result_randomforestclassifier > best_result_randomforestclassifier:
            best_model_randomforestclassifier = model_randomforestclassifier # сохраним наилучшую модель
            best_result_randomforestclassifier = model_randomforestclassifier.score(features_valid, target_valid)#  сохраним наилучшее значение метрики accuracy на валидационных данных
            best_depth = depth 
            best_n_estimators = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result_randomforestclassifier,  "с гиперпараметром max_depth: =",best_depth, "и с n_estimators:=", 
     best_n_estimators)

Accuracy наилучшей модели на валидационной выборке: 0.80248833592535 с гиперпараметром max_depth: = 8 и с n_estimators:= 8


**`LogisticRegression`**

**Ещё один алгоритм машинного обучения - логистическая регрессия.
В логистической регрессии параметров мало. Что-либо вызубрить по признакам в формуле не выйдет, поэтому и вероятность переобучения невелика.**

In [11]:
model_logisticregression = LogisticRegression(random_state=12345)
model_logisticregression.fit(features_train, target_train) # обучим модель
predictions_logisticregression = model_logisticregression.predict(features_valid) # получим предсказания модели
result_logisticregression = accuracy_score(predictions_logisticregression, target_valid) # посчитаем качество модели
        
print("Accuracy модели:", result_logisticregression)

Accuracy модели: 0.7107309486780715


**Вывод:**
- Accuracy лучшей модели `DecisionTreeClassifier`: 0.7853810264385692 с гиперпараметром max_depth: = 3

- Accuracy наилучшей модели `RandomForestClassifier`  на валидационной выборке: 0.80248833592535 с гиперпараметром max_depth: = 8 и с n_estimators:= 8

- Accuracy модели  `LogisticRegression` : 0.7107309486780715

## Проверьте модель на тестовой выборке

Наилучашую Accuracy на валидационной выборке показала модель `RandomForestClassifier` : 0.80248833592535 с гиперпараметрами max_depth: = 8 и с n_estimators:= 8. 

Проверим эту же модель на тестовой выборке.

In [12]:
model_randomforestclassifier = RandomForestClassifier(random_state=12345, n_estimators=8, max_depth=8) # обучим модель с заданным количеством деревьев
model_randomforestclassifier.fit(features_train, target_train) # обучим модель на обучающей выборке
predictions_randomforestclassifier = model_randomforestclassifier.predict(features_test)
result_randomforestclassifier = model_randomforestclassifier.score(features_test, target_test) # посчитаем качество модели на тестовой выборке

print("Accuracy модели на тестовой выборке:", result_randomforestclassifier)

Accuracy модели на тестовой выборке: 0.7962674961119751


Accuracy на тестовой выборке  модель `RandomForestClassifier` показала: 0.7962674961119751 с гиперпараметрами max_depth: = 8 и с n_estimators:= 8. 

Это может быть связано с тем, что модель переобучена на обучающей выборке, то есть модель хорошо работает на данных, на которых она обучалась, но не так хорошо работает на новых данных. 

## (бонус) Проверьте модели на адекватность

**Для проверки адекватности модели машинного обучения в Python, удобно использовать DummyClassifier из библиотеки scikit-learn. Это простой классификатор, который можно использовать в качестве эталонной модели.**

In [13]:
clf = DummyClassifier(strategy='most_frequent')
clf.fit(features_train, target_train)
result_clf = clf.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_clf)

Accuracy DummyClassifier: 0.7060653188180405


In [14]:
print("Accuracy модели RandomForestClassifier на валидационной выборке:", best_result_randomforestclassifier)

Accuracy модели RandomForestClassifier на валидационной выборке: 0.80248833592535


Можно сказать, что модель RandomForestClassifier адекватна и эффективна.

**Итоговый вывод:**
- Изучили наши данные, предобработка данных не требовалась — уже была сделана.


- В результате разбиения данных на выборки мы получили: 
    - Обучающую выборку размером 60% от исходных данных.
    - Валидационную выборку размером 20% от исходных данных.
    - Тестовую выборку размером 20% от исходных данных.
    
    
    
- Исследовали модели и получили такие результаты: 
  -  Accuracy лучшей модели DecisionTreeClassifier: 0.7853810264385692 с гиперпараметром max_depth: = 3
  - Accuracy наилучшей модели RandomForestClassifier на валидационной выборке: 0.80248833592535 с гиперпараметром max_depth: = 8 и с n_estimators:= 8
  - Accuracy модели LogisticRegression : 0.7107309486780715
    
    
- Проверили модель на тестовой выборке:
  - Accuracy на тестовой выборке модель RandomForestClassifier показала: 0.7962674961119751 с гиперпараметрами max_depth: = 8 и с n_estimators:= 8. Это может быть связано с тем, что модель переобучена на обучающей выборке, то есть модель хорошо работает на данных, на которых она обучалась, но не так хорошо работает на новых данных.



- Проверили модель на адекватность: 
     - Для проверки адекватности модели машинного обучения в Python использовали DummyClassifier из библиотеки scikit-learn. 
        - Можно сказать, что модель RandomForestClassifier адекватна и эффективна.

